In [4]:
#Importa bibliotecas utilizadas
import tensorflow as tf
import numpy as np
import time as tm
import math
import sys
import pickle as pkl
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 


###start here
"""
Parameters
K - size of x
N - size of y
snrdb_low - the lower bound of noise db used during training
snr_high - the higher bound of noise db used during training
L - number of layers in DetNet
v_size = size of auxiliary variable at each layer
hl_size - size of hidden layer at each DetNet layer (the dimention the layers input are increased to
startingLearningRate - the initial step size of the gradient descent algorithm
decay_factor & decay_step_size - each decay_step_size steps the learning rate decay by decay_factor
train_iter - number of train iterations
train_batch_size - batch size during training phase
test_iter - number of test iterations
test_batch_size  - batch size during testing phase
LOG_LOSS - equal 1 if loss of each layer should be sumed in proportion to the layer depth, otherwise all losses have the same weight 
res_alpha- the proportion of the previuos layer output to be added to the current layers output (view ResNet article)
snrdb_low_test & snrdb_high_test & num_snr - when testing, num_snr different SNR values will be tested, uniformly spread between snrdb_low_test and snrdb_high_test 
By Neev Samuel neev(dot)samuel(at)gmail(dot)com
"""
#Inicio de sessão
sess = tf.compat.v1.InteractiveSession()

#Parâmetros
K = 20
N = 30
snrdb_low = 7.0
snrdb_high = 14.0
snr_low = 10.0 ** (snrdb_low/10.0)
snr_high = 10.0 ** (snrdb_high/10.0)
L=90
v_size = 2*K
hl_size = 8*K
startingLearningRate = 0.0001
decay_factor = 0.97
decay_step_size = 1000
train_iter = 20000
train_batch_size = 5000
test_iter= 200
test_batch_size = 1000
LOG_LOSS = 1
res_alpha=0.9
num_snr = 6
snrdb_low_test=8.0
snrdb_high_test=13.0

"""Data generation for train and test phases
In this example, both functions are the same.
This duplication is in order to easily allow testing cases where the test is over different distributions of data than in the training phase.
e.g. training over gaussian i.i.d. channels and testing over a specific constant channel.
currently both test and train are over i.i.d gaussian channel.
"""

#Geração de dados para teste
def generate_data_iid_test(B,K,N,snr_low,snr_high):
    H_=np.random.randn(B,N,K)
    W_=np.zeros([B,K,K])
    x_=np.sign(np.random.rand(B,K)-0.5)
    y_=np.zeros([B,N])
    w=np.random.randn(B,N)
    Hy_=x_*0
    HH_=np.zeros([B,K,K])
    SNR_= np.zeros([B])
    for i in range(B):
        SNR = np.random.uniform(low=snr_low,high=snr_high)
        H=H_[i,:,:]
        tmp_snr=(H.T.dot(H)).trace()/K
        H_[i,:,:]=H
        y_[i,:]=(H.dot(x_[i,:])+w[i,:]*np.sqrt(tmp_snr)/np.sqrt(SNR))
        Hy_[i,:]=H.T.dot(y_[i,:])
        HH_[i,:,:]=H.T.dot( H_[i,:,:])
        SNR_[i] = SNR
    return y_,H_,Hy_,HH_,x_,SNR_

#Gerar dados de treinamento
def generate_data_train(B,K,N,snr_low,snr_high):
    H_=np.random.randn(B,N,K)
    W_=np.zeros([B,K,K])
    x_=np.sign(np.random.rand(B,K)-0.5)
    y_=np.zeros([B,N])
    w=np.random.randn(B,N)
    Hy_=x_*0
    HH_=np.zeros([B,K,K])
    SNR_= np.zeros([B])
    for i in range(B):
        SNR = np.random.uniform(low=snr_low,high=snr_high)
        H=H_[i,:,:]
        tmp_snr=(H.T.dot(H)).trace()/K
        H_[i,:,:]=H
        y_[i,:]=(H.dot(x_[i,:])+w[i,:]*np.sqrt(tmp_snr)/np.sqrt(SNR))
        Hy_[i,:]=H.T.dot(y_[i,:])
        HH_[i,:,:]=H.T.dot( H_[i,:,:])
        SNR_[i] = SNR
    return y_,H_,Hy_,HH_,x_,SNR_

#Normalização da variável y 
def piecewise_linear_soft_sign(x):
    #Criação de um vetor unidimensional de valor 0.1
    t = tf.Variable(0.1)
    
    #Escalonamento/padronização da variável y (label)
    y = -1+tf.nn.relu(x+t)/(tf.abs(t)+0.00001)-tf.nn.relu(x-t)/(tf.abs(t)+0.00001)
    
    return y

#Função que preenche as camadas com dado ruidoso
def affine_layer(x,input_size,output_size,Layer_num):
    W = tf.Variable(tf.random_normal([input_size, output_size], stddev=0.01))
    w = tf.Variable(tf.random_normal([1, output_size], stddev=0.01))
    y = tf.matmul(x, W)+w
    return y

#Escalonando as layers 
def relu_layer(x,input_size,output_size,Layer_num):
    y = tf.nn.relu(affine_layer(x,input_size,output_size,Layer_num))
    return y

#Normalizando os labels
def sign_layer(x,input_size,output_size,Layer_num):
    y = piecewise_linear_soft_sign(affine_layer(x,input_size,output_size,Layer_num))
    return y

#Criando vetores matrizes que serão utilizados na fase de treinamento
HY = tf.placeholder(tf.float32,shape=[None,K])
X = tf.placeholder(tf.float32,shape=[None,K])
HH = tf.placeholder(tf.float32,shape=[None, K , K])

#Incialização de variáveis
batch_size = tf.shape(HY)[0]

#???
X_LS = tf.matmul(tf.expand_dims(HY,1),tf.matrix_inverse(HH))
X_LS= tf.squeeze(X_LS,1)
loss_LS = tf.reduce_mean(tf.square(X - X_LS))
ber_LS = tf.reduce_mean(tf.cast(tf.not_equal(X,tf.sign(X_LS)), tf.float32))

#Inicializando vetores p/ armazenar o histórico de dados
S=[]
S.append(tf.zeros([batch_size,K]))
V=[]
V.append(tf.zeros([batch_size,v_size]))
LOSS=[]
LOSS.append(tf.zeros([]))
BER=[]
BER.append(tf.zeros([]))


#The architecture of DetNet
#Calculado LOSS e BER
for i in range(1,L):
    temp1 = tf.matmul(tf.expand_dims(S[-1],1),HH)
    temp1= tf.squeeze(temp1,1)
    Z = tf.concat([HY,S[-1],temp1,V[-1]],1)
    ZZ = relu_layer(Z,3*K + v_size , hl_size,'relu'+str(i))
    S.append(sign_layer(ZZ , hl_size , K,'sign'+str(i)))
    S[i]=(1-res_alpha)*S[i]+res_alpha*S[i-1]
    V.append(affine_layer(ZZ , hl_size , v_size,'aff'+str(i)))
    V[i]=(1-res_alpha)*V[i]+res_alpha*V[i-1]  
    if LOG_LOSS == 1:
        LOSS.append(np.log(i)*tf.reduce_mean(tf.reduce_mean(tf.square(X - S[-1]),1)/tf.reduce_mean(tf.square(X - X_LS),1)))
    else:
        LOSS.append(tf.reduce_mean(tf.reduce_mean(tf.square(X - S[-1]),1)/tf.reduce_mean(tf.square(X - X_LS),1)))          
    BER.append(tf.reduce_mean(tf.cast(tf.not_equal(X,tf.sign(S[-1])), tf.float32)))

TOTAL_LOSS=tf.add_n(LOSS)

#Inicializando variáveis p/ treinamento
global_step = tf.Variable(0, trainable=False)
learning_rate = tf.train.exponential_decay(startingLearningRate, global_step, decay_step_size, decay_factor, staircase=True)
train_step = tf.train.AdamOptimizer(learning_rate).minimize(TOTAL_LOSS)
init_op=tf.initialize_all_variables()

sess.run(init_op)
#Training DetNet
for i in range(train_iter): 
    #Geração de dados de treinamento
    batch_Y, batch_H, batch_HY, batch_HH, batch_X , SNR1= generate_data_train(train_batch_size,K,N,snr_low,snr_high)
    
    #Atualizando as variáveis, de forma que possam ser tratadas. (variáveis originais)
    train_step.run(feed_dict={HY: batch_HY, HH: batch_HH, X: batch_X})
    
    if i % 100 == 0 :
        #Gerados dados de treinamento 
        batch_Y, batch_H, batch_HY, batch_HH, batch_X ,SNR1= generate_data_iid_test(train_batch_size,K,N,snr_low,snr_high)
        results = sess.run([loss_LS,LOSS[L-1],ber_LS,BER[L-1]], {HY: batch_HY, HH: batch_HH, X: batch_X})
        print_string = [i]+results
        print ((' ').join('%s' % x for x in print_string))
          

#Testing the trained model
snrdb_list = np.linspace(snrdb_low_test,snrdb_high_test,num_snr)
#Converte p/ db (log)
snr_list = 10.0 ** (snrdb_list/10.0)

#Inicialização de variáveis
bers = np.zeros((1,num_snr))
times = np.zeros((1,num_snr))
tmp_bers = np.zeros((1,test_iter))
tmp_times = np.zeros((1,test_iter))

#Loop
for j in range(num_snr):
    for jj in range(test_iter):
        print('snr:')
        print(snrdb_list[j])
        print('test iteration:')
        print(jj)
        
        #Atualiza as variáveis com os valores obtidos a partir da função
        batch_Y, batch_H, batch_HY, batch_HH, batch_X ,SNR1= generate_data_iid_test(test_batch_size , K,N,snr_list[j],snr_list[j])
        tic = tm.time()
        
        #Organzizando em vetor a BER p/ cada coluna do vetor BER. Considerando cada iteração
        tmp_bers[:,jj] = np.array(sess.run(BER[L-1], {HY: batch_HY, HH: batch_HH, X: batch_X}))    
        toc = tm.time()
        tmp_times[0][jj] =toc - tic
        
        
    bers[0][j] = np.mean(tmp_bers,1)
    times[0][j] = np.mean(tmp_times[0])/test_batch_size

print('snrdb_list')
print(snrdb_list)
print('bers')
print(bers)
print('times')
print(times)

C:\Users\josin\Anaconda3\lib\site-packages\tensorflow_core\python\client\session.py:1752: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


0 0.26757988 27.83119 0.02926 0.46415
100 0.27025804 7.1343336 0.02996 0.08034
200 0.2663382 2.3626783 0.02789 0.01704
300 0.26245052 1.5245081 0.02745 0.01089
400 0.26272678 1.0036936 0.02689 0.00726
500 0.26635462 0.70714915 0.02849 0.00627
600 0.26945505 0.51749265 0.02839 0.00535
700 0.26776707 0.40081564 0.02851 0.00452
800 0.2646637 0.34232032 0.02863 0.00456
900 0.2623548 0.3093388 0.02722 0.00437
1000 0.2710303 0.28107765 0.02916 0.00489
1100 0.26828536 0.29855347 0.0294 0.00461
1200 0.25909138 0.26723966 0.02684 0.00437
1300 0.2698899 0.25164548 0.02972 0.00474
1400 0.26447818 0.2901662 0.02742 0.00515
1500 0.26081997 0.24624853 0.02741 0.00454
1600 0.26615953 0.2419968 0.02842 0.00522
1700 0.26481304 0.24219778 0.02868 0.00515
1800 0.2623977 0.23651706 0.0278 0.00501
1900 0.26635304 0.2326442 0.02873 0.00493
2000 0.27158397 0.2525449 0.03021 0.00536
2100 0.262535 0.2109434 0.02936 0.00428
2200 0.26221412 0.21796572 0.02828 0.00513
2300 0.2666295 0.24036002 0.02827 0.00528
240

19200 0.2594312 0.18312936 0.02779 0.00456
19300 0.2697398 0.19356006 0.02957 0.00484
19400 0.2640817 0.18019383 0.02901 0.00462
19500 0.26509073 0.18851392 0.02845 0.00482
19600 0.2646596 0.18783054 0.02786 0.00474
19700 0.26647544 0.19122978 0.02883 0.00486
19800 0.2688646 0.19719124 0.02897 0.00498
19900 0.271786 0.16968147 0.02951 0.00433
snr:
8.0
test iteration:
0
snr:
8.0
test iteration:
1
snr:
8.0
test iteration:
2
snr:
8.0
test iteration:
3
snr:
8.0
test iteration:
4
snr:
8.0
test iteration:
5
snr:
8.0
test iteration:
6
snr:
8.0
test iteration:
7
snr:
8.0
test iteration:
8
snr:
8.0
test iteration:
9
snr:
8.0
test iteration:
10
snr:
8.0
test iteration:
11
snr:
8.0
test iteration:
12
snr:
8.0
test iteration:
13
snr:
8.0
test iteration:
14
snr:
8.0
test iteration:
15
snr:
8.0
test iteration:
16
snr:
8.0
test iteration:
17
snr:
8.0
test iteration:
18
snr:
8.0
test iteration:
19
snr:
8.0
test iteration:
20
snr:
8.0
test iteration:
21
snr:
8.0
test iteration:
22
snr:
8.0
test iterati

snr:
9.0
test iteration:
78
snr:
9.0
test iteration:
79
snr:
9.0
test iteration:
80
snr:
9.0
test iteration:
81
snr:
9.0
test iteration:
82
snr:
9.0
test iteration:
83
snr:
9.0
test iteration:
84
snr:
9.0
test iteration:
85
snr:
9.0
test iteration:
86
snr:
9.0
test iteration:
87
snr:
9.0
test iteration:
88
snr:
9.0
test iteration:
89
snr:
9.0
test iteration:
90
snr:
9.0
test iteration:
91
snr:
9.0
test iteration:
92
snr:
9.0
test iteration:
93
snr:
9.0
test iteration:
94
snr:
9.0
test iteration:
95
snr:
9.0
test iteration:
96
snr:
9.0
test iteration:
97
snr:
9.0
test iteration:
98
snr:
9.0
test iteration:
99
snr:
9.0
test iteration:
100
snr:
9.0
test iteration:
101
snr:
9.0
test iteration:
102
snr:
9.0
test iteration:
103
snr:
9.0
test iteration:
104
snr:
9.0
test iteration:
105
snr:
9.0
test iteration:
106
snr:
9.0
test iteration:
107
snr:
9.0
test iteration:
108
snr:
9.0
test iteration:
109
snr:
9.0
test iteration:
110
snr:
9.0
test iteration:
111
snr:
9.0
test iteration:
112
snr:
9.

snr:
10.0
test iteration:
160
snr:
10.0
test iteration:
161
snr:
10.0
test iteration:
162
snr:
10.0
test iteration:
163
snr:
10.0
test iteration:
164
snr:
10.0
test iteration:
165
snr:
10.0
test iteration:
166
snr:
10.0
test iteration:
167
snr:
10.0
test iteration:
168
snr:
10.0
test iteration:
169
snr:
10.0
test iteration:
170
snr:
10.0
test iteration:
171
snr:
10.0
test iteration:
172
snr:
10.0
test iteration:
173
snr:
10.0
test iteration:
174
snr:
10.0
test iteration:
175
snr:
10.0
test iteration:
176
snr:
10.0
test iteration:
177
snr:
10.0
test iteration:
178
snr:
10.0
test iteration:
179
snr:
10.0
test iteration:
180
snr:
10.0
test iteration:
181
snr:
10.0
test iteration:
182
snr:
10.0
test iteration:
183
snr:
10.0
test iteration:
184
snr:
10.0
test iteration:
185
snr:
10.0
test iteration:
186
snr:
10.0
test iteration:
187
snr:
10.0
test iteration:
188
snr:
10.0
test iteration:
189
snr:
10.0
test iteration:
190
snr:
10.0
test iteration:
191
snr:
10.0
test iteration:
192
snr:
10.0


snr:
12.0
test iteration:
39
snr:
12.0
test iteration:
40
snr:
12.0
test iteration:
41
snr:
12.0
test iteration:
42
snr:
12.0
test iteration:
43
snr:
12.0
test iteration:
44
snr:
12.0
test iteration:
45
snr:
12.0
test iteration:
46
snr:
12.0
test iteration:
47
snr:
12.0
test iteration:
48
snr:
12.0
test iteration:
49
snr:
12.0
test iteration:
50
snr:
12.0
test iteration:
51
snr:
12.0
test iteration:
52
snr:
12.0
test iteration:
53
snr:
12.0
test iteration:
54
snr:
12.0
test iteration:
55
snr:
12.0
test iteration:
56
snr:
12.0
test iteration:
57
snr:
12.0
test iteration:
58
snr:
12.0
test iteration:
59
snr:
12.0
test iteration:
60
snr:
12.0
test iteration:
61
snr:
12.0
test iteration:
62
snr:
12.0
test iteration:
63
snr:
12.0
test iteration:
64
snr:
12.0
test iteration:
65
snr:
12.0
test iteration:
66
snr:
12.0
test iteration:
67
snr:
12.0
test iteration:
68
snr:
12.0
test iteration:
69
snr:
12.0
test iteration:
70
snr:
12.0
test iteration:
71
snr:
12.0
test iteration:
72
snr:
12.0
test

snr:
13.0
test iteration:
118
snr:
13.0
test iteration:
119
snr:
13.0
test iteration:
120
snr:
13.0
test iteration:
121
snr:
13.0
test iteration:
122
snr:
13.0
test iteration:
123
snr:
13.0
test iteration:
124
snr:
13.0
test iteration:
125
snr:
13.0
test iteration:
126
snr:
13.0
test iteration:
127
snr:
13.0
test iteration:
128
snr:
13.0
test iteration:
129
snr:
13.0
test iteration:
130
snr:
13.0
test iteration:
131
snr:
13.0
test iteration:
132
snr:
13.0
test iteration:
133
snr:
13.0
test iteration:
134
snr:
13.0
test iteration:
135
snr:
13.0
test iteration:
136
snr:
13.0
test iteration:
137
snr:
13.0
test iteration:
138
snr:
13.0
test iteration:
139
snr:
13.0
test iteration:
140
snr:
13.0
test iteration:
141
snr:
13.0
test iteration:
142
snr:
13.0
test iteration:
143
snr:
13.0
test iteration:
144
snr:
13.0
test iteration:
145
snr:
13.0
test iteration:
146
snr:
13.0
test iteration:
147
snr:
13.0
test iteration:
148
snr:
13.0
test iteration:
149
snr:
13.0
test iteration:
150
snr:
13.0
